# Demonstration of PatentTools

PatentTools is a collection of basic tools for fetching and interpreting patent information. It currently supports only US Patents.

Its basic input is a US Patent number. It provides bibliographic information about the patent (title, assignee, etc.). 

## Fetch and parse the patent

In [1]:
lookup_num = "US 8,622,391"

In [2]:
from patenttools import USPTOLookup

In [3]:
patent = USPTOLookup(lookup_num)

In [4]:
patent.title

'Self-contained dice shaker system '

In [5]:
patent.assignee

'IGT (Reno, NV)'

In [6]:
patent.filing_date

'January 3, 2013'

In [7]:
for i in range(0, 5):
    print(patent.cited_nums[i], patent.cited_links[i])

2109736 /netacgi/nph-Parser?Sect2=PTO1&Sect2=HITOFF&p=1&u=%2Fnetahtml%2FPTO%2Fsearch-bool.html&r=1&f=G&l=50&d=PALL&RefSrch=yes&Query=PN%2F2109736
2302333 /netacgi/nph-Parser?Sect2=PTO1&Sect2=HITOFF&p=1&u=%2Fnetahtml%2FPTO%2Fsearch-bool.html&r=1&f=G&l=50&d=PALL&RefSrch=yes&Query=PN%2F2302333
2619349 /netacgi/nph-Parser?Sect2=PTO1&Sect2=HITOFF&p=1&u=%2Fnetahtml%2FPTO%2Fsearch-bool.html&r=1&f=G&l=50&d=PALL&RefSrch=yes&Query=PN%2F2619349
2693962 /netacgi/nph-Parser?Sect2=PTO1&Sect2=HITOFF&p=1&u=%2Fnetahtml%2FPTO%2Fsearch-bool.html&r=1&f=G&l=50&d=PALL&RefSrch=yes&Query=PN%2F2693962
3508755 /netacgi/nph-Parser?Sect2=PTO1&Sect2=HITOFF&p=1&u=%2Fnetahtml%2FPTO%2Fsearch-bool.html&r=1&f=G&l=50&d=PALL&RefSrch=yes&Query=PN%2F3508755


In [8]:
patent.claims[0]

'1. A method of operating a self-contained dice shaker assembly, the dice shaker assembly including one or more dice, the method comprising: detecting, using one or more electronic sensors of the self-contained dice shaker assembly, motion associated with the dice; determining, using one or more electronic processors of the self-contained dice shaker assembly, whether the detected motion satisfies one or more predetermined motion parameters; and transmitting, a communication device of the self-contained dice shaker assembly, information related to the detected motion to an external device.'

## The TextDistiller class.

In conjunction with an instance of the TextDistiller class, PatentTools may highlight sentences may be pulled from the claims and/or description sections; highlights are selected based on word frequencies. 

In [9]:
from patenttools.distiller import TextDistiller

In [10]:
claims_distiller = TextDistiller(" ".join(patent.claims))

By default, three sentences are selected to highlight. The results are returned as a list:

In [11]:
claims_distiller.highlights

['A method of operating a self-contained dice shaker assembly, the dice shaker assembly including one or more dice, the method comprising: detecting, using one or more electronic sensors of the self-contained dice shaker assembly, motion associated with the dice; determining, using one or more electronic processors of the self-contained dice shaker assembly, whether the detected motion satisfies one or more predetermined motion parameters; and transmitting, a communication device of the self-contained dice shaker assembly, information related to the detected motion to an external device.',
 'A method of operating a self-contained dice shaker assembly, the dice shaker assembly including one or more dice, the method comprising: detecting, using one or more electronic sensors of the self-contained dice shaker assembly, motion associated with the dice; monitoring, using one or more electronic processors of the self-contained dice shaker assembly, a shake quality during a shaking event by d

In [12]:
desc_distiller = TextDistiller(patent.description)

Users may specify an alternative number of highlights, too:

In [13]:
desc_distiller.gen_highlights(num_top_sentences = 2)

['The dice shaker assembly illustrated in FIG.',
 'Dice shaker assembly 100 includes dice receptacle 104, cover member 108, and sensor assembly 112.']

## The PatentTree class.

Instances of the PatentTree class are used to understand the broader significance of a patent in the context of the other patents that it cites. Instantiating this class with a patent number automatically looks up and parses *all the S Patents cited by the user-specified patent*. Due to rate limits for this request, this process can take several minutes.

In [14]:
from patenttools import PatentTree

In [15]:
parent_patent = PatentTree(lookup_num)

Reading cited patents. This will take some time...
Hit a server-imposed rate limit. Waiting 10 s.


In [16]:
parent_patent.most_cited_assignees

[('IGT (Reno, NV)', 6),
 ('Monson; Marvin (Denver, CO)', 1),
 ('Woodall, Jr.; William E. (Benson, NC)', 1),
 ('Other', 26)]

The entire body of cited US Patent literature is combined into a single corpus:

In [17]:
parent_patent.cited_corpus[:500]

' The present invention relates to amusement devices and to games of chance, and more particularly to a dice shaker assembly and die channels therefor that are provided with means to prevent a die from lodging as it moves between the extreme ends of the die channel. \n\nBACKGROUND OF THE INVENTION \n\nIn U.S.  Pat.  No. 3,820,792, I disclosed a dice shaker assembly that could be used in a variety of dice games, and which was particularly designed to be used in games such as "Liars Poker." Basically t'

The combined corpus can then be supplied to the TextDistiller class to generate bigrams to provide a sense for the important features of the patented works.

In [18]:
cited_distiller = TextDistiller(parent_patent.cited_corpus)

By default, 25 bigrams are returned with a window size of 3 and a frequency filter of 5:

In [19]:
cited_distiller.bigrams

[('chuck', 'luck'),
 ('reversed', 'etched'),
 ('pai', 'gow'),
 ('private', 'branch'),
 ('tilt', 'tip'),
 ('japanese', 'laid'),
 ('look', 'feel'),
 ('mixing', 'dispensing'),
 ('appended', 'claim'),
 ('j', 'k'),
 ('neon', 'lamp'),
 ('regional', 'office'),
 ('hierarchy', 'graphic'),
 ('air', 'bubble'),
 ('accelerometer', 'tilt'),
 ('codec', 'dsp'),
 ('void', 'cutout'),
 ('pitch', 'degree'),
 ('departing', 'spirit'),
 ('branch', 'exchange'),
 ('household', 'book'),
 ('human', 'discernable'),
 ('corporate', 'headquarters'),
 ('varying', 'degree'),
 ('fwdarw', 'fwdarw')]

Alternative window sizes and frequency filters may also be supplied:

In [20]:
more_bigrams = cited_distiller.gen_bigrams(win_size = 5, freq_filter = 100)
more_bigrams

[('playing', 'piece'),
 ('gaming', 'element'),
 ('portable', 'telephone'),
 ('shaker', 'assembly'),
 ('communication', 'link'),
 ('dice', 'shaker'),
 ('remote', 'communication'),
 ('portable', 'communication'),
 ('communication', 'interface'),
 ('dice', 'assembly'),
 ('portable', 'device'),
 ('communication', 'device'),
 ('remote', 'device'),
 ('communication', 'remote'),
 ('shake', 'dice'),
 ('communication', 'communication'),
 ('dice', 'game'),
 ('device', 'device'),
 ('device', 'communication'),
 ('one', 'dice'),
 ('dice', 'dice'),
 ('dice', 'one')]